In [9]:
import librosa
import numpy as np
import pandas as pd
import joblib
from tensorflow.keras.models import load_model
from sklearn.preprocessing import StandardScaler

In [10]:
def extract_features(y, sr, n_mfcc=40):
    try:
        S = np.abs(librosa.stft(y))
        contrast = librosa.feature.spectral_contrast(S=S, sr=sr)
        harm, perc = librosa.effects.hpss(y)

        features = {
            'chroma_stft_mean': np.mean(librosa.feature.chroma_stft(y=y, sr=sr)),
            'rms_mean': np.mean(librosa.feature.rms(y=y)),
            'spectral_centroid_mean': np.mean(librosa.feature.spectral_centroid(y=y, sr=sr)),
            'spectral_bandwidth_mean': np.mean(librosa.feature.spectral_bandwidth(y=y, sr=sr)),
            'spectral_rolloff_mean': np.mean(librosa.feature.spectral_rolloff(y=y, sr=sr)),
            'spectral_contrast_mean': np.mean(contrast),
            'harmonic_mean': np.mean(harm),
            'percussive_mean': np.mean(perc),
            'zero_crossing_rate_mean': np.mean(librosa.feature.zero_crossing_rate(y=y))
        }
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
        for i, mfcc in enumerate(mfccs):
            features[f'mfcc{i+1}_mean'] = np.mean(mfcc)
        feature_values = np.array(list(features.values())).reshape(1, -1)
        return feature_values
    except Exception as e:
        print(f"Error extracting features: {e}")
        return np.zeros((1, 49))  # Return a row of zeros if there is an error


In [11]:
def predict_song_genre(file_path, model, scaler, segment_duration=30):
    y, sr = librosa.load(file_path, sr=None)  # Load the whole song
    song_duration = librosa.get_duration(y=y, sr=sr)
    predictions = []
    
    for start in np.arange(0, song_duration, segment_duration):
        end = start + segment_duration
        if end > song_duration:
            end = song_duration
        y_segment = y[int(start * sr):int(end * sr)]
        features = extract_features(y_segment, sr)
        features_scaled = scaler.transform(features)
        prediction = model.predict(features_scaled)
        predicted_class_index = np.argmax(prediction)
        predictions.append(predicted_class_index)
        
    # Analyze the results for all segments
    from collections import Counter
    most_common = Counter(predictions).most_common(1)
    print(f"Most predicted class for {file_path} is: {most_common[0][0]} with {most_common[0][1]} out of {len(predictions)} segments")
    return predictions

In [12]:
# โหลดโมเดลและ scaler
model = load_model('best_model_DNN.h5')
scaler = joblib.load('Model_DNN_scaler_.pkl')

c:\Users\Rapee\Desktop\P4\Python\python-08-tryvercel\venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\Rapee\Desktop\P4\Python\python-08-tryvercel\venv\Lib\site-packages\keras\src\optimizers\base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(
c:\Users\Rapee\Desktop\P4\Python\python-08-tryvercel\venv\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 0.24.2 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings

In [14]:
# Example usage
file_path = 'uploads/1812 overture - Tchaikovsky.wav'
predict_song_genre(file_path, model, scaler)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━

[3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3]